In [61]:
import tensorflow as tf
import sklearn

In [62]:
tf.test.is_gpu_available()

True

In [63]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import scipy.io.wavfile as wav
import numpy as np
import librosa
from python_speech_features import mfcc

# Generación de ficheros

In [64]:
import numpy as np
import os
def generate_file_data(dir,name):
    directory=dir
    a={'0':'cero','1':'uno','2':'dos','3':'tres','4':'cuatro','5':'cinco','6':'seis','7':'siete','8':'ocho','9':'nueve'}
    da=os.listdir(directory)
    da.sort()
    file = open(dir+name+'.txt',"w")
    for filename in da:
        if '.wav' in filename:
            file.write(filename+','+a[filename[0]]+'\n')
    file.close() 
    with open(directory+'/'+name+'.txt') as f:
        read_data = f.read()
        f.closed
    read_data=read_data.split('\n')
    read_data=read_data[0:len(read_data)-1]
    return read_data

# Encoding words with One Hot Encoding

In [65]:
vocabulary_words=['cero','uno','dos','tres','cuatro','cinco','seis','siete','ocho','nueve']
label_encoder=LabelEncoder()
integer_encoded = label_encoder.fit_transform(vocabulary_words)
print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse=False,categories='auto')
integer_encoded = integer_encoded.reshape(-1, 1)
onehot_encoded=onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[0 9 3 8 2 1 6 7 5 4]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [66]:
def encode(x):# will transform the label using one hot encoding/take a group of strings

    integers=label_encoder.transform(x)
    integers=integers.reshape(-1,1)
    return onehot_encoder.transform(integers)

In [67]:
x=encode(['cero','uno'])
print(x)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [68]:
def decode(x):
    inverted = onehot_encoder.inverse_transform(x)
    y=inverted.astype(dtype=int)# castea los valores
    y=y.reshape(1,-1)
    y=y[0]
    y=label_encoder.inverse_transform(y)
    return y

In [69]:
def to_text(m,mfcc):
    label=m.predict_classes(mfcc)
    text=label_encoder.inverse_transform(label)
    text=text[0]
    return text

In [70]:
y=decode(x)
print(y)

['cero' 'uno']


# MFCC 


In [93]:
def mfcc_features(DIR,list_dir):
    mfcc_audios=[]
    for dir in list_dir:
        wave, sr = librosa.load(DIR+dir, mono=True)
        features= librosa.feature.mfcc(wave, sr,n_mfcc=20)
        features = sklearn.preprocessing.scale(features, axis=1)
        #features = mfcc(audio, samplerate=fs)
        try:
            features=np.pad(features,((0,0),(0,160-len(features[0]))),mode='constant', constant_values=0)
        except OSError as err:
            print(dir)
            #print("Error OS: {0}".format(err))
        
        #print(features)
        #print(features.shape)
        mfcc_audios.append(features)
        #print(mfcc_audios)
    #print(mfcc_audios)
    mfcc_audios=np.array(mfcc_audios)
    return mfcc_audios

In [94]:
def prepare_data(dir,name):
    file = open(dir+name)
    f=file.read()
    file.close()
    f=f.split('\n')
    f=f[0:len(f)-1]
    labels=[]
    names_audios=[]
    for i in f:
        j=i.split(',')
        names_audios.append(j[0])
        labels.append(j[1])
    labels=np.array(labels)
    onehot_encoded= encode(labels)
    #print(len(onehot_encoded))
    mfcc=mfcc_features(dir,names_audios)
    #print(onehot_encoded)
    #d=decode(onehot_encoded)
    #print(d)
    print(name+' OK')
    return mfcc,onehot_encoded



In [95]:
class dataset:
    def __init__(self,data):
        self.i=0#para el shuffle
        self.data_dir=data
        self.shuffle=None
        self.dir_training=data+'/training/'
        self.dir_test=data+'/test/'
        self.training_set=None
        self.test_set=None
    def split_dataset(self):
        generate_file_data(self.dir_training,name='training')
        generate_file_data(self.dir_test,name='test')
        self.training_set=prepare_data(self.dir_training,'training.txt')
        self.test_set=prepare_data(self.dir_test,'test.txt')
    #def next_batch(self,size):
    #
    #    return self.training_set[self.shuffle]
    

In [96]:
d=dataset('data')
d.split_dataset()

training.txt OK
test.txt OK


In [97]:
d.dir_test

'data/test/'

In [98]:
d.training_set

(array([[[-0.91332112, -0.88884379, -0.81044921, ...,  0.        ,
           0.        ,  0.        ],
         [-1.20854392, -1.11266398, -0.81749539, ...,  0.        ,
           0.        ,  0.        ],
         [-0.19381587, -0.04986061,  0.34608203, ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.09185654, -0.4501449 , -0.66089419, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.4319833 ,  0.08398442, -0.02356124, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.45809148, -0.00812885, -0.03912434, ...,  0.        ,
           0.        ,  0.        ]],
 
        [[-1.52812778, -1.21816265, -0.98880818, ...,  0.        ,
           0.        ,  0.        ],
         [-0.84560769, -0.29826789,  0.00949638, ...,  0.        ,
           0.        ,  0.        ],
         [-0.5518056 , -0.06760515, -0.05673965, ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.13594544,  0.3354751

In [99]:
len(d.training_set[0])

270

In [ ]:
#[d.training_set[0][i].shape for i in range(len(d.training_set[0]))] #will display numofcep

In [100]:
np.size(d.training_set[0])

864000

In [101]:
d.test_set[0]

array([[[-2.05998940e+00, -1.23003454e+00, -8.28596864e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.59875744e+00,  2.71496056e-01,  1.41003877e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.53973475e+00,  3.73814923e+00,  4.68282822e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [-2.97762685e-01, -1.32114334e-01, -6.11860095e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 6.06772767e-01,  4.56950189e-01,  5.16246237e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-5.81951387e-01, -1.06613079e+00, -4.76588902e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-1.38449484e+00, -1.24522622e+00,  5.31944850e-02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.16117738e-01,  3.48829661e-01,  9.04523696e-01, ...,
          0.00000000e+00,  0.00000000e

In [102]:
d.test_set[1]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 

# Red neuronal 

In [103]:
trainX, trainY = d.training_set[0],d.training_set[1]
testX, testY = d.test_set[0],d.test_set[1]


In [104]:
import sklearn

In [105]:
trainX=np.matrix.transpose(trainX,[0,2,1])
testX=np.matrix.transpose(testX,[0,2,1])

In [106]:
trainX

array([[[-0.91332112, -1.20854392, -0.19381587, ...,  0.09185654,
          0.4319833 ,  0.45809148],
        [-0.88884379, -1.11266398, -0.04986061, ..., -0.4501449 ,
          0.08398442, -0.00812885],
        [-0.81044921, -0.81749539,  0.34608203, ..., -0.66089419,
         -0.02356124, -0.03912434],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-1.52812778, -0.84560769, -0.5518056 , ...,  0.13594544,
          0.79682373,  0.61725581],
        [-1.21816265, -0.29826789, -0.06760515, ...,  0.33547512,
          1.3434033 ,  1.62400325],
        [-0.98880818,  0.00949638, -0.05673965, ...,  0.40121452,
          0.50865569,  0.90935095],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [107]:
testX

array([[[-2.05998940e+00, -2.59875744e+00, -1.53973475e+00, ...,
         -2.97762685e-01,  6.06772767e-01, -5.81951387e-01],
        [-1.23003454e+00,  2.71496056e-01,  3.73814923e+00, ...,
         -1.32114334e-01,  4.56950189e-01, -1.06613079e+00],
        [-8.28596864e-01,  1.41003877e+00,  4.68282822e+00, ...,
         -6.11860095e-01,  5.16246237e-01, -4.76588902e-01],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-1.38449484e+00,  3.16117738e-01,  8.53240544e-01, ...,
         -5.97427469e-01,  4.92634723e-01, -6.54543818e-01],
        [-1.24522622e+00,  3.48829661e-01,  7.18403011e-01, ...,
         -6.20955057e-01,  4.06928454e

In [108]:
import tensorflow as tf
import time 
n_units=128
time_steps=160
n_inputs=20
batch_size=10
n_epochs=10
n_class=10

In [109]:
model=tf.keras.Sequential()
#model.add(tf.layers.Dropout(0.5))
model.add(tf.keras.layers.LSTM(n_units, input_shape=(time_steps,n_inputs)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.layers.Dense(n_class, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [110]:
ti=time.time()
for i in range(100):
    result=model.fit(trainX,trainY,batch_size=batch_size,epochs=n_epochs,validation_data=[testX,testY])
    model.save('Models/keras_training')
tiempofinal=time.time()

Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 10s 37ms/step - loss: 2.3045 - acc: 0.0667 - val_loss: 2.3053 - val_acc: 0.1000
Epoch 2/10
270/270 [==============================] - 6s 23ms/step - loss: 2.3041 - acc: 0.0741 - val_loss: 2.3055 - val_acc: 0.1000
Epoch 3/10
270/270 [==============================] - 6s 23ms/step - loss: 2.3048 - acc: 0.0963 - val_loss: 2.3049 - val_acc: 0.1000
Epoch 4/10
270/270 [==============================] - 6s 23ms/step - loss: 2.3041 - acc: 0.1148 - val_loss: 2.3049 - val_acc: 0.1000
Epoch 5/10
270/270 [==============================] - 7s 25ms/step - loss: 2.3042 - acc: 0.0926 - val_loss: 2.3047 - val_acc: 0.1000
Epoch 6/10
270/270 [==============================] - 6s 23ms/step - loss: 2.3024 - acc: 0.1037 - val_loss: 2.3047 - val_acc: 0.1000
Epoch 7/10
270/270 [==============================] - 6s 24ms/step - loss: 2.3047 - acc: 0.1148 - val_loss: 2.3047 - val_acc: 0.1000
Epoch 8/10
270/270 [===

270/270 [==============================] - 6s 22ms/step - loss: 2.3062 - acc: 0.0889 - val_loss: 2.3006 - val_acc: 0.1333
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 24ms/step - loss: 2.3040 - acc: 0.0926 - val_loss: 2.3006 - val_acc: 0.1333
Epoch 2/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3032 - acc: 0.0667 - val_loss: 2.3005 - val_acc: 0.1000
Epoch 3/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3033 - acc: 0.0852 - val_loss: 2.3006 - val_acc: 0.1000
Epoch 4/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3017 - acc: 0.1037 - val_loss: 2.3004 - val_acc: 0.1000
Epoch 5/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3044 - acc: 0.0778 - val_loss: 2.3003 - val_acc: 0.1000
Epoch 6/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3033 - acc: 0.0963 - val_loss: 2.3001 - val_acc: 0.1333
Epoch 7/10
270/270 [===============

270/270 [==============================] - 6s 21ms/step - loss: 2.3017 - acc: 0.1074 - val_loss: 2.3035 - val_acc: 0.1000
Epoch 10/10
270/270 [==============================] - 6s 21ms/step - loss: 2.3018 - acc: 0.0963 - val_loss: 2.3028 - val_acc: 0.0667
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 21ms/step - loss: 2.3050 - acc: 0.0778 - val_loss: 2.3031 - val_acc: 0.0667
Epoch 2/10
270/270 [==============================] - 6s 21ms/step - loss: 2.3038 - acc: 0.0889 - val_loss: 2.3037 - val_acc: 0.1000
Epoch 3/10
270/270 [==============================] - 6s 21ms/step - loss: 2.3037 - acc: 0.1185 - val_loss: 2.3037 - val_acc: 0.1000
Epoch 4/10
270/270 [==============================] - 6s 21ms/step - loss: 2.3046 - acc: 0.0778 - val_loss: 2.3038 - val_acc: 0.1000
Epoch 5/10
270/270 [==============================] - 6s 21ms/step - loss: 2.3023 - acc: 0.0889 - val_loss: 2.3040 - val_acc: 0.1667
Epoch 6/10
270/270 [==============

270/270 [==============================] - 6s 22ms/step - loss: 2.3043 - acc: 0.1074 - val_loss: 2.2976 - val_acc: 0.1333
Epoch 9/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3036 - acc: 0.0963 - val_loss: 2.2968 - val_acc: 0.0667
Epoch 10/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3040 - acc: 0.1037 - val_loss: 2.2970 - val_acc: 0.1000
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3039 - acc: 0.0852 - val_loss: 2.2972 - val_acc: 0.0667
Epoch 2/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3043 - acc: 0.0889 - val_loss: 2.2973 - val_acc: 0.1000
Epoch 3/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3047 - acc: 0.0815 - val_loss: 2.2974 - val_acc: 0.1333
Epoch 4/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3032 - acc: 0.0963 - val_loss: 2.2974 - val_acc: 0.1333
Epoch 5/10
270/270 [==============

270/270 [==============================] - 6s 22ms/step - loss: 2.3044 - acc: 0.0778 - val_loss: 2.3001 - val_acc: 0.1667
Epoch 8/10
270/270 [==============================] - 7s 24ms/step - loss: 2.3057 - acc: 0.0852 - val_loss: 2.3000 - val_acc: 0.1000
Epoch 9/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3023 - acc: 0.1111 - val_loss: 2.3002 - val_acc: 0.1000
Epoch 10/10
270/270 [==============================] - 6s 22ms/step - loss: 2.3043 - acc: 0.1222 - val_loss: 2.3002 - val_acc: 0.1000
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 23ms/step - loss: 2.3043 - acc: 0.0889 - val_loss: 2.3000 - val_acc: 0.1333
Epoch 2/10
270/270 [==============================] - 6s 21ms/step - loss: 2.3032 - acc: 0.1296 - val_loss: 2.2996 - val_acc: 0.1667
Epoch 3/10
270/270 [==============================] - 6s 21ms/step - loss: 2.3041 - acc: 0.1111 - val_loss: 2.3069 - val_acc: 0.0333
Epoch 4/10
270/270 [==============

...

270/270 [==============================] - 6s 22ms/step - loss: 0.1072 - acc: 0.9778 - val_loss: 8.4311 - val_acc: 0.1333
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 21ms/step - loss: 0.1235 - acc: 0.9667 - val_loss: 8.5647 - val_acc: 0.1000
Epoch 2/10
270/270 [==============================] - 6s 21ms/step - loss: 0.1095 - acc: 0.9741 - val_loss: 8.8447 - val_acc: 0.1000
Epoch 3/10
270/270 [==============================] - 6s 22ms/step - loss: 0.0875 - acc: 0.9889 - val_loss: 9.1175 - val_acc: 0.1000
Epoch 4/10
270/270 [==============================] - 6s 23ms/step - loss: 0.1240 - acc: 0.9593 - val_loss: 9.0054 - val_acc: 0.1667
Epoch 5/10
270/270 [==============================] - 6s 23ms/step - loss: 0.4826 - acc: 0.7963 - val_loss: 7.6806 - val_acc: 0.2000
Epoch 6/10
270/270 [==============================] - 6s 22ms/step - loss: 0.6844 - acc: 0.7852 - val_loss: 8.3650 - val_acc: 0.1333
Epoch 7/10
270/270 [===============

270/270 [==============================] - 6s 22ms/step - loss: 0.1759 - acc: 0.9407 - val_loss: 8.7760 - val_acc: 0.1333
Epoch 10/10
270/270 [==============================] - 6s 21ms/step - loss: 0.1519 - acc: 0.9519 - val_loss: 9.2727 - val_acc: 0.1333
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 22ms/step - loss: 0.1375 - acc: 0.9519 - val_loss: 9.6756 - val_acc: 0.1000
Epoch 2/10
270/270 [==============================] - 6s 21ms/step - loss: 0.0735 - acc: 0.9815 - val_loss: 9.4046 - val_acc: 0.0667
Epoch 3/10
270/270 [==============================] - 6s 21ms/step - loss: 0.0550 - acc: 0.9889 - val_loss: 9.5936 - val_acc: 0.1000
Epoch 4/10
270/270 [==============================] - 6s 21ms/step - loss: 0.1031 - acc: 0.9815 - val_loss: 9.4322 - val_acc: 0.1000
Epoch 5/10
270/270 [==============================] - 6s 21ms/step - loss: 0.1341 - acc: 0.9667 - val_loss: 8.8615 - val_acc: 0.1333
Epoch 6/10
270/270 [==============

270/270 [==============================] - 7s 25ms/step - loss: 0.0361 - acc: 0.9926 - val_loss: 9.0635 - val_acc: 0.1333
Epoch 9/10
270/270 [==============================] - 6s 22ms/step - loss: 0.0349 - acc: 0.9926 - val_loss: 9.0678 - val_acc: 0.1667
Epoch 10/10
270/270 [==============================] - 6s 22ms/step - loss: 0.0445 - acc: 0.9852 - val_loss: 9.4355 - val_acc: 0.1333
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 22ms/step - loss: 0.0553 - acc: 0.9889 - val_loss: 8.7879 - val_acc: 0.1667
Epoch 2/10
270/270 [==============================] - 6s 22ms/step - loss: 0.0460 - acc: 0.9852 - val_loss: 9.0795 - val_acc: 0.1667
Epoch 3/10
270/270 [==============================] - 6s 22ms/step - loss: 0.1085 - acc: 0.9704 - val_loss: 9.4592 - val_acc: 0.1000
Epoch 4/10
270/270 [==============================] - 6s 21ms/step - loss: 0.3787 - acc: 0.8630 - val_loss: 9.4177 - val_acc: 0.0667
Epoch 5/10
270/270 [==============

270/270 [==============================] - 6s 22ms/step - loss: 0.1908 - acc: 0.9407 - val_loss: 8.6930 - val_acc: 0.1667
Epoch 8/10
270/270 [==============================] - 6s 21ms/step - loss: 0.2269 - acc: 0.9296 - val_loss: 9.1106 - val_acc: 0.1000
Epoch 9/10
270/270 [==============================] - 6s 21ms/step - loss: 0.1677 - acc: 0.9444 - val_loss: 9.4770 - val_acc: 0.0667
Epoch 10/10
270/270 [==============================] - 6s 22ms/step - loss: 0.1097 - acc: 0.9593 - val_loss: 8.3967 - val_acc: 0.1333
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 22ms/step - loss: 0.0814 - acc: 0.9741 - val_loss: 9.1049 - val_acc: 0.1000
Epoch 2/10
270/270 [==============================] - 6s 22ms/step - loss: 0.0770 - acc: 0.9852 - val_loss: 9.4014 - val_acc: 0.1000
Epoch 3/10
270/270 [==============================] - 7s 28ms/step - loss: 0.0773 - acc: 0.9778 - val_loss: 8.9879 - val_acc: 0.1000
Epoch 4/10
270/270 [==============

270/270 [==============================] - 6s 22ms/step - loss: 1.0951 - acc: 0.6667 - val_loss: 7.6607 - val_acc: 0.0667
Epoch 7/10
270/270 [==============================] - 6s 22ms/step - loss: 0.7118 - acc: 0.8333 - val_loss: 7.5914 - val_acc: 0.1333
Epoch 8/10
270/270 [==============================] - 6s 21ms/step - loss: 0.3975 - acc: 0.9000 - val_loss: 7.5191 - val_acc: 0.1333
Epoch 9/10
270/270 [==============================] - 6s 21ms/step - loss: 0.3827 - acc: 0.9000 - val_loss: 7.7896 - val_acc: 0.0333
Epoch 10/10
270/270 [==============================] - 6s 22ms/step - loss: 0.3804 - acc: 0.9037 - val_loss: 7.9717 - val_acc: 0.0667
Train on 270 samples, validate on 30 samples
Epoch 1/10
270/270 [==============================] - 6s 22ms/step - loss: 0.3616 - acc: 0.9037 - val_loss: 7.9884 - val_acc: 0.0667
Epoch 2/10
270/270 [==============================] - 6s 22ms/step - loss: 0.2457 - acc: 0.9444 - val_loss: 7.8595 - val_acc: 0.0000e+00
Epoch 3/10
270/270 [==========

In [111]:
print((tiempofinal-ti)/60)

99.39352280298868


In [112]:
trainX[0]

array([[-0.91332112, -1.20854392, -0.19381587, ...,  0.09185654,
         0.4319833 ,  0.45809148],
       [-0.88884379, -1.11266398, -0.04986061, ..., -0.4501449 ,
         0.08398442, -0.00812885],
       [-0.81044921, -0.81749539,  0.34608203, ..., -0.66089419,
        -0.02356124, -0.03912434],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [113]:
y=model.predict([[trainX[0]]])

In [114]:
model.predict_classes([[trainX[0]]])

array([0])

In [116]:
label_encoder.inverse_transform([0])

array(['cero'], dtype='<U6')

In [117]:
to_text(model,[[trainX[0]]])

'cero'

In [118]:
y

array([[8.5458267e-01, 1.3367206e-05, 1.1010848e-04, 1.2222943e-06,
        1.5461472e-05, 3.2351786e-04, 6.4432106e-06, 1.4490633e-01,
        3.8986556e-05, 1.9558286e-06]], dtype=float32)

In [182]:
trainY[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [120]:
class_result=y[0]>=y[0][np.argmax(y,axis=-1)]

In [192]:
y[0][9]

0.8600664

In [121]:
class_result

array([ True, False, False, False, False, False, False, False, False,
       False])

In [122]:
decode([class_result])

array(['cero'], dtype='<U6')

In [202]:
class_result=class_result.astype('int')

In [214]:
import librosa
wave, sr = librosa.load('data/training/3_nicolas_18.wav', mono=True)
features= librosa.feature.mfcc(wave, sr,n_mfcc=20)

In [242]:
wave

array([-0.00721617, -0.00877718, -0.00902262, ...,  0.00046424,
        0.00275924,  0.        ], dtype=float32)

In [215]:
features = sklearn.preprocessing.scale(features, axis=1)
features=np.pad(features,((0,0),(0,100-len(features[0]))),mode='constant', constant_values=0)

In [217]:
f=np.matrix.transpose(np.array([features]),[0,2,1])

In [219]:
label=m.predict_classes(f)

In [220]:
label_encoder.inverse_transform(label)

array(['three'], dtype='<U5')

In [225]:
to_text(m,f)

'three'

# Load Model 

In [2]:
import tensorflow as tf
m=tf.keras.models.load_model('Models/keras_training')

In [4]:
from keras.utils import plot_model
plot_model(m, to_file='model.png')

In [208]:
m.predict_classes([[trainX[0]]])

array([9])

# Recording audios and predict them

In [232]:
import sounddevice as sd
import ipywidgets as widgets
from IPython.display import display

In [233]:
button = widgets.Button(description="Record")
display(button)

def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)


Button(description='Record', style=ButtonStyle())

In [240]:
duration = 10000  # seconds
fs=16000
myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=1)

In [241]:
myrecording

array([[-3.3615484e-07],
       [ 8.7624744e-07],
       [-9.1520860e-07],
       ...,
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00]], dtype=float32)

In [139]:
import pyaudio
import wave
 
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 2
WAVE_OUTPUT_FILENAME = "file.wav"
 
audio = pyaudio.PyAudio()
 
# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
print ("recording...")
frames = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print ("finished recording")
 
 
# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

recording...
finished recording


In [287]:
wave

<module 'wave' from '/home/visoc/anaconda3/envs/tf/lib/python3.6/wave.py'>

In [140]:
import librosa
wave, sr = librosa.load('file.wav', mono=True)
features= librosa.feature.mfcc(wave, sr,n_mfcc=20)

In [141]:
wave

array([-0.00195091, -0.0033551 , -0.00449437, ...,  0.00154216,
        0.0010393 ,  0.        ], dtype=float32)

In [142]:
features = sklearn.preprocessing.scale(features, axis=1)
features=np.pad(features,((0,0),(0,160-len(features[0]))),mode='constant', constant_values=0)
f=np.matrix.transpose(np.array([features]),[0,2,1])

In [143]:
label=model.predict_classes(f)

In [144]:
label

array([9])

In [145]:
label_encoder.inverse_transform(label)

array(['uno'], dtype='<U6')